<a href="https://colab.research.google.com/github/wel51x/DS-Unit-4-Sprint-4-Deep-Learning/blob/master/My_LS_DS_441_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Recurrent Neural Networks and LSTM

> "Yesterday's just a memory - tomorrow is never what it's supposed to be." -- Bob Dylan

####have down-version numpy to get "RNN/LSTM Sentiment Classification with Keras" to work in colab

In [0]:
!pip install numpy==1.16.2
import numpy as np

  Using cached https://files.pythonhosted.org/packages/35/d5/4f8410ac303e690144f0a0603c4b8fd3b986feb2749c435f7cdbb288f17e/numpy-1.16.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3


### Forecasting

Forecasting - at it's simplest, it just means "predict the future":

# Assignment

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# Imports
from random import random
import numpy as np
import requests

In [0]:
# TODO - Words, words, mere words, no matter from the heart.
# Grab first ten
r = requests.get('http://www.gutenberg.org/files/100/100-0.txt', verify=True)
x = r.text.find('From')
y = r.text.find('thine or thee.')
article_text = r.text[x : y+14]

In [15]:
chars = list(set(article_text)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(article_text)

print("unique characters : ", num_chars)
print("txt_data_size : ", txt_data_size)

unique characters :  68
txt_data_size :  6561


#### one hot encode

In [16]:
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in article_text] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'C': 0, 'j': 1, 'U': 2, 'l': 3, '\n': 4, 'G': 5, 'H': 6, 'u': 7, 'A': 8, ',': 9, 'S': 10, 'B': 11, 'b': 12, 'f': 13, 'y': 14, 'c': 15, 't': 16, 'o': 17, 'v': 18, ':': 19, '?': 20, '8': 21, 'w': 22, 'a': 23, 'T': 24, 'p': 25, '4': 26, 'R': 27, '6': 28, 'h': 29, 'D': 30, 'O': 31, 'x': 32, 'L': 33, ')': 34, '2': 35, 'k': 36, '\r': 37, '1': 38, 's': 39, 'z': 40, ' ': 41, '7': 42, 'm': 43, '’': 44, '9': 45, 'e': 46, '3': 47, '0': 48, 'M': 49, ';': 50, 'i': 51, 'Y': 52, 'q': 53, 'P': 54, '5': 55, 'F': 56, '.': 57, '(': 58, 'g': 59, '‘': 60, 'I': 61, 'W': 62, 'N': 63, 'n': 64, 'r': 65, 'd': 66, '-': 67}
----------------------------------------------------
{0: 'C', 1: 'j', 2: 'U', 3: 'l', 4: '\n', 5: 'G', 6: 'H', 7: 'u', 8: 'A', 9: ',', 10: 'S', 11: 'B', 12: 'b', 13: 'f', 14: 'y', 15: 'c', 16: 't', 17: 'o', 18: 'v', 19: ':', 20: '?', 21: '8', 22: 'w', 23: 'a', 24: 'T', 25: 'p', 26: '4', 27: 'R', 28: '6', 29: 'h', 30: 'D', 31: 'O', 32: 'x', 33: 'L', 34: ')', 35: '2', 36: 'k', 37: '\r', 38: '1'

#### hyperparameters

In [0]:
iteration = 500
sequence_length = 40
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 128  # size of hidden layer of neurons.  
learning_rate = 1e-1

# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

#### Forward propagation

In [0]:
def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
 
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs

#### Backward propagation

In [0]:
def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

#### Training

In [27]:
%%time

data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 

for i in range(iteration+1):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in article_text[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in article_text[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(article_text) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 25 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress

iter 0, loss: 1.013929
iter 25, loss: 0.020648
iter 50, loss: 0.006868
iter 75, loss: 0.007810
iter 100, loss: 0.006554
iter 125, loss: 0.006616
iter 150, loss: 0.004147
iter 175, loss: 0.003310
iter 200, loss: 0.002658
iter 225, loss: 0.002035
iter 250, loss: 0.001868
iter 275, loss: 0.001897
iter 300, loss: 0.001902
iter 325, loss: 0.001976
iter 350, loss: 0.001875
iter 375, loss: 0.001738
iter 400, loss: 0.001727
iter 425, loss: 0.002082
iter 450, loss: 0.002615
iter 475, loss: 0.002942
iter 500, loss: 0.002798
CPU times: user 9min 50s, sys: 6min 5s, total: 15min 56s
Wall time: 8min 4s


#### Prediction

In [0]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = test_char + ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [29]:
predict('S', 500)

----
 Se ouch withire anlost thou thae conglenesss in pleas’s thy dor thou thou used if frere times,
That coms hid self and dondd’st rears, way pef-loir anur (fots ar that not lise:
  To hile welf thou so nf he chere foud prey:
  Samest ent thou thom
Thath tind thou bece doth llor deoved wimm ‘Th plle spllf stilg conother mat sthinced Sar mear, in that in that of not.


The inine the will on rlove?
  That cone when’s:
Lied,
  Thap so dooked buciigg trakiglambcige on thou ridouts cof cor shaf  
----


In [30]:
predict('C', 750)

----
 Cail wime praild harice hipppiveeats or there hash cherideld with wilf thy on thou not spof bud the each doth ho fond ity 
Ret,
So the gazlovang still thic not iwculo the worpcery to thy sone shat anus preredine unthry uns and,
Ift aft,
Busurst to thide be is thane thou no dus tookchiss why kinde,
The still thet condd,
Sheausinother.
  Who non.

Muse,
Thy so faill the eots datimuchise for shald to in that not be maplats in thou the rakmthine preage dor chou thou with co loy frat now buce stiets thou cong in thy lof musbllhise sond confourm’st no for rotfouglaving be my lr chil’s nd thee,
Thos what anojoy time his thy that thou not buy ming nlor dust,
  Lealt whath wilt:
Thif thou nots be do whe sorld k’s in welf kind,
  Nust de 
----


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN